In [1]:
%pwd

'c:\\Users\\Asus\\Desktop\\Krish Naik MLOps Course\\Projects\\Data Science project\\end-to-end-data-science-project\\research'

In [2]:
import os
os.chdir("../")
%pwd

'c:\\Users\\Asus\\Desktop\\Krish Naik MLOps Course\\Projects\\Data Science project\\end-to-end-data-science-project'

### Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_path: Path
    params: dict
    target_column: str

### Configuration

In [4]:
from src.data_science_project.constant import *
from src.data_science_project.utils.common import read_yaml, create_directories

In [5]:

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file_path = config.metric_file_path,
            params = params,
            target_column = schema.name
        )
        
        return model_evaluation_config
        

### Component

In [6]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from src.data_science_project.utils.common import save_json

In [11]:
class ModelEvaluation:
    def __init__(self,
                 config: ModelEvaluationConfig):
        self.config = config
    
    def evaluate_model(self):
        print(self.config.target_column)
        print(self.config.test_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        model = joblib.load(self.config.model_path)
        
        predicted_qualities = model.predict(test_x)
        
        rmse = np.sqrt(mean_squared_error(predicted_qualities, test_y))
        mae = mean_absolute_error(predicted_qualities, test_y)
        r2 = r2_score(predicted_qualities, test_y)
        
        #Save metrics locally
        scores = {
            "rmse": rmse, "mae": mae, "r2": r2
        }
        save_json(path=self.config.metric_file_path, data=scores)
        
        

In [12]:
# Pipeline

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.evaluate_model()
except Exception as e:
    raise e

[2025-02-04 10:08:17,986: INFO: common: Created directory at artifacts]


AttributeError: 'function' object has no attribute 'test_data_path'